In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np

import qubex.pulse as qp
from qubex.simulator import Control, Coupling, QuantumSimulator, QuantumSystem, Transmon

In [ ]:
transmons = [
    Transmon(
        label="Q08",
        dimension=3,
        frequency=7.646,
        anharmonicity=-0.356,
        relaxation_rate=0.00005,
        dephasing_rate=0.00005,
    ),
    Transmon(
        label="Q09",
        dimension=3,
        frequency=8.442,
        anharmonicity=-0.448,
        relaxation_rate=0.00005,
        dephasing_rate=0.00005,
    ),
]

couplings = [
    Coupling(pair=("Q08", "Q09"), strength=0.01),
]

system = QuantumSystem(
    objects=transmons,
    couplings=couplings,
)

In [ ]:
system.hamiltonian

In [ ]:
simulator = QuantumSimulator(system)

In [ ]:
initial_state = system.state(
    {
        "Q08": "0",
        "Q09": "0",
    },
)

In [ ]:
duration = 30
tau = 10

rx180 = qp.FlatTop(
    duration=duration,
    amplitude=np.pi / (duration - tau),
    tau=tau,
)
ry180 = rx180.shifted(np.pi / 2)

with qp.PulseSchedule(["Q08", "Q09"]) as ps:
    ps.add("Q08", rx180)
    ps.barrier()
    ps.add("Q08", ry180)
    ps.add("Q09", rx180)
    ps.add("Q09", ry180)

ps.plot()

waveforms = ps.get_sampled_sequences()

In [ ]:
target = transmons[0]

detuning = 0.001

dt = qp.get_sampling_period()

control_0 = Control(
    target="Q08",
    frequency=transmons[0].frequency + detuning,
    rabi_rates=waveforms["Q08"],
    durations=[dt] * len(waveforms["Q08"]),
)
control_0.plot()

control_1 = Control(
    target="Q09",
    frequency=transmons[1].frequency + detuning,
    rabi_rates=waveforms["Q09"],
    durations=[dt] * len(waveforms["Q09"]),
)
control_1.plot()

In [ ]:
result = simulator.mesolve(
    controls=[control_0, control_1],
    initial_state=initial_state,
)

In [ ]:
final_state = result.states[-1]
final_state

In [ ]:
final_state_target = result.get_substates("Q08")[-1]
final_state_target

In [ ]:
result.plot_population_dynamics()
result.plot_population_dynamics("Q08")
result.plot_population_dynamics("Q09")

In [ ]:
result.get_substates("Q08")

In [ ]:
result.show_last_population("Q08")

In [ ]:
result.display_bloch_sphere("Q08")
result.display_bloch_sphere("Q09")